The general idea behind this is we want to be able to do some basic queries on ODS data. For example we may want a list of GP's who work at

In [29]:
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
import pandas as pd

headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

url = 'https://files.digital.nhs.uk/assets/ods/current/egpcur.zip'
response = urlopen(url=url,  timeout=120)

myzip = ZipFile(BytesIO(response.read()))
myzip.namelist()
myzip.extractall('ZIP')

df = pd.read_csv('ZIP/egpcur.csv', header=None, index_col=False, names=["GMP","Name",3,4,5,6,7,8,9,10,11,12,13,14,"PRAC_ODS",16,17,18,19,20,21,22])

df



C:\Users\lilyh\AppData\Local\Temp\ipykernel_8828\253352844.py:15: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ZIP/egpcur.csv', header=None, index_col=False, names=["GMP","Name",3,4,5,6,7,8,9,10,11,12,13,14,"PRAC_ODS",16,17,18,19,20,21,22])
C:\Users\lilyh\AppData\Local\Temp\ipykernel_8828\253352844.py:15: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv('ZIP/egpcur.csv', header=None, index_col=False, names=["GMP","Name",3,4,5,6,7,8,9,10,11,12,13,14,"PRAC_ODS",16,17,18,19,20,21,22])


,GMP,Name,3,4,5,6,7,8,9,10,...,13,14,PRAC_ODS,16,17,18,19,20,21,22
0,G0102005,ALLEN EB,Y11,QAL,"FIRCROFT, LONDON ROAD",ENGLEFIELD GREEN,EGHAM,SURREY,NaN,TW20 0BS,...,C,P,H81600,19740401,19910401.0,NaN,NaN,NaN,NaN,1
1,G0102926,ANDERSON MG,Y61,QUE,LENSFIELD MEDICAL PRAC.,48 LENSFIELD ROAD,CAMBRIDGE,CAMBRIDGESHIRE,NaN,CB2 1EH,...,C,O,D81001,19740401,19911231.0,01223 651020,NaN,NaN,NaN,1
2,G0105912,ADLER S,Y56,QMJ,682 FINCHLEY ROAD,GOLDERS GREEN,LONDON,NaN,NaN,NW11 7NP,...,C,O,E83600,19740401,19920731.0,020 84559994,NaN,NaN,NaN,1
3,G0107031,ATTWOOD DC,Y62,QOP,GREAT LEVER HEALTH CENTRE,"RUPERT STREET,GREAT LEVER",BOLTON,LANCASHIRE,NaN,BL3 6RN,...,C,O,P82013,19740401,19910525.0,01204 462141,NaN,NaN,NaN,1
4,G0107725,ALEXANDER PJ,Y01,QDF,10 WEST END,SWANLAND,HUMBERSIDE,NaN,NaN,HU14 3PE,...,C,P,B81600,19740401,19940409.0,0482 633570,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123768,G9996043,UNIDENTIFIED GPS,W00,Q99,NORTH WALES HA,PRESWYLFA,HENDY ROAD,MOLD FLINTSHIRE,NaN,CH7 1PZ,...,C,P,W94699,19910401,20140930.0,NaN,NaN,NaN,NaN,1
123769,G9996050,UNIDENTIFIED GPS,W00,Q99,MORGANNWG HA,41 HIGH STREET,SWANSEA,WEST GLAMORGAN,NaN,SA1 1LT,...,C,P,W95699,19910401,20140930.0,NaN,NaN,NaN,NaN,1
123770,G9996067,COMMITTEES LOCUM,W00,QW3,DEPUTISING SERVICES,POWYS,NaN,NaN,NaN,NaN,...,C,P,W96699,19910401,19950331.0,NaN,NaN,NaN,NaN,1
123771,G9996074,COMMITTEES LOCUM,W00,QW2,DEPUTISING SERVICES,SOUTH-GLAMORGAN,NaN,NaN,NaN,NaN,...,C,P,W97699,19910401,19950331.0,NaN,NaN,NaN,NaN,1
